# 1. Simple Integer Arithmetic Problem


### 1.1 Problem Statement
Write a program that can parse a simple integer arithmetic expression to give an integer result

### 1.2 Example
"1 + 2 * 3" should yield the answer 7.

### 1.3 Setup
You must be able to run the program such that you can provide the name of a file with one expression per line and for each line output "<expr> = <answer>".
    
### 1.4 Conditions
The expression should support operators +, -, *, /, integers, parentheses.

Implement and test your solution to illustrate how you would implement production code.

You may use 3rd party libraries to implement parts of your solution (e.g. a parser library) but you should not use a 3rd party library that provides the entire solution.

You may implement this in any language, ideally .NET, with F#, but any is fine
### 1.5 Submission
Please submit via GitHub or a similar repository from which the solution can be downloaded.

# 2. Solution Statement

The ask is to support a subset of all available mathematical operators. 

## 2.1 Operator Precedence

In mathematics, the order of the operators in an expression is important. For example:

- expression: 1 + 2 * 3

With no formal rule on operator precedence, this could be evaluated in two ways:

- method 1: Add 1 to 2 to give 3, then multiple 3 by 3. Answer 9
- method 2: Multiple 2 by 3 to give 6, then add 1. Answer 7.

Method 2 is the correct approach as it follows the <b>operator precedence</b> rule, which is in itself a collection of rules that state the order precedence. They are as follows:
    
- exponentiation and root extraction
- multiplication and division
- addition and subtraction

## 2.2 Expression Notation

In the expression example we are given 1 + 2 * 3. This notation is called <b>infix</b>, an expression of the form a op b. When an operator is between every pair of operands. 
This is the traditional way we create mathematical expressions, but it is difficult for a computer to use when handling operator precedence and dealing with parenthesis. A more suitable notation is called <b>postfix</b> expression. The <b>postfix</b> expression is of the form a b op. When an operator is followed for every pair of operands. This notation is easier todeal with as it allows the <b>operands</b> and <b>operators</b> to be added in stack like structures in a specific order. This means that "1 + 1" becomes "1 1 +"

## 2.3 Expression Conversion

The mechanism to create a postfix notation from an infix notation was developed by <b>Edsger Dijkstra</b> in 1961 and can be found in the following paper https://www.cs.utexas.edu/~EWD/MCReps/MR35.PDF

As previously mentioned, this approach uses stacks to reverse the order of the operators in the expression.

### 2.4.1 Assumptions

- The infix notation is parseable and in a consistent format
- The expression generator adds a single space between all values, operators and parenthesis

# 3 Solution Implementation 

The following operator dictionary servers dual purposes. It will return the operator precedence when asked and will also provide a lambda function which is ultimately used to calculate the integer values. The ask is to support a subset of all available mathematical operators (*,/,+,-).

In [4]:
ops = {
    '*': {'precedence': 2,'op': lambda x,y: x * y},
    '/': {'precedence': 2,'op': lambda x,y: x / y},
    '+': {'precedence': 1,'op': lambda x,y: x + y},
    '-': {'precedence': 1,'op': lambda x,y: x -  y},
    '(': {'precedence': 0},
}

### 3.1 Shunting Yard Algorithm

This is the algorithm I will use to perform the expression translation to <b>postfix</b>. The details of the algorithm are illuminated in the comments that describe each step. Essentially it is forward traversal through all the tokens in the expression string, assigning them to either an operator stack or an output stack (holds the operands). Then, using the operator precedence rule some operators and popped to the end of the output stack and this is also the case when dealing with parenthesis, as they have a higher precedence than the four operators we are using. This gives the algorithm its "shunting yard" name as it is similar to the method in which trains and moved to seperate tracks. I have also added a check to ensure there are matching opening/closing parenthesis.

In [3]:
LH = '(' 
RH = ')'

def algo(expression, debug = False):
    
    out_stack = []
    ops_stack = []
    
    if debug:
        print('INFIX:', expression)
    
    # Check matching parenthesis
    lhs = sum(c == '(' for c in expression)
    rhs = sum(c == ')' for c in expression)
    if lhs != rhs:
        raise Exception('Unable to parse INFIX due to unmatched parenthesis')
    
    # While there are expression tokens to be read
    for token in expression.split(' '):
        
        # If it's a number add it to the output stack
        if token.lstrip("-").isdigit():
            out_stack.append(token)
            
        # If it's a left bracket
        elif token == LH:
            # Push it onto the operator stack
            ops_stack.append(token)
            
        # If it's a right bracket
        elif token == RH:
            # While there's not a left bracket at the top of the stack:
            while len(ops_stack) > 0 and ops_stack[-1] != LH:
                # Pop operators from the operator stack onto the output stack
                out_stack.append(ops_stack.pop())
            # Pop the left bracket from the operator stack and discard it
            ops_stack.pop()
            
        # If it's an operator
        elif token in ops:
            # While there's an operator on the top of the operator stack with greater precedence
            while len(ops_stack) > 0 and ops[ops_stack[-1]]['precedence'] >= ops[token]['precedence']:
                # Pop operators from the operator stack onto the output stack
                out_stack.append(ops_stack.pop())
            # Push the current operator onto the stack
            ops_stack.append(token)
            
    # While there are operators on the operator stack, pop them to the output stack       
    while ops_stack:
        out_stack.append(ops_stack.pop())
    return out_stack

### 3.2 Postfix Evaluator

Finally we need a means to calculate the integer result from our postfix notation. This is trivial as we now simply pop the values from the output and if we have an operator and pair of operands the value is calculated. This value is then placed onto the result stack. If we discover an operand in the traversal this is simply placed back onto the result stack in anticipation of its correct operator. This ensures the operators act upon the correct operands as established by the shunting-yard algo.

In [7]:
def evaluate(expression, debug=False):
    if debug:
        print('POSTFIX: %s' % ' '.join(expression))
    stack = []
    for token in expression:
        if token in ops:
            arg2 = stack.pop()
            arg1 = stack.pop()
            print(arg1, ' - ', arg2)
            result = ops[token]['op'](arg1, arg2)
            stack.append(result)
        else:
            stack.append(int(token))
    return stack.pop()